# Gemini Sentiment Analysis Agent Demo


### Get API keys

Before you can use the Gemini API, you must first obtain an API key. If you don't already have one, create a key with one click in [Google AI Studio](https://makersuite.google.com/app/apikey).  You'll also need an [OpenAI API key](https://openai.com/).

Once you have your API keys create the following Google Colab "secrets"
`GOOGLE_API_KEY` and `OPENAI_API_KEY` which will contain your respective keys.

## Initial Setup

In [1]:
!pip install llama-index google-generativeai openai pypdf -q

In [5]:
!pip install ipywidgets -q

In [4]:
!pip install trulens_eval -q

In [6]:
import os
import openai

openai.api_key = os.environ.get('OPENAI_API_KEY')

In [7]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO, force=True)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

### Import from LlamaIndex and TruLens

In [8]:
# from trulens_eval import Feedback, Tru, TruLlama
# from trulens_eval.feedback import Groundedness
# from trulens_eval.feedback.provider.openai import OpenAI

# tru = Tru()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


### Vectorize Mindfulness Guides

In [9]:
import os.path
from llama_index import (
  VectorStoreIndex,
  SimpleDirectoryReader,
  StorageContext,
  load_index_from_storage,
  )

# Attempt to load embeddings
try:
  storage_context = StorageContext.from_defaults(
      persist_dir="./storage/challenging_child"
  )
  challenging_child_index = load_index_from_storage(storage_context)

  storage_context = StorageContext.from_defaults(
      persist_dir="./storage/mindfulness_TB_50"
  )
  mindfulness_TB_50_index = load_index_from_storage(storage_context)

  storage_context = StorageContext.from_defaults(
      persist_dir="./storage/mindfulness_TB_relationships"
  )
  mindfulness_TB_relationships_index = load_index_from_storage(storage_context)
except:
  # Need to generate embeddings
  challenging_child_docs = SimpleDirectoryReader(
      input_files=["./data/The_Challenging_Child_Toolbox.pdf"]
  ).load_data(show_progress=True)
  mindfulness_TB_50_docs = SimpleDirectoryReader(
      input_files=["./data/The_Mindfulness_Toolbox__50_Practical_Tips_Tools.pdf"]
  ).load_data(show_progress=True)
  mindfulness_TB_relationships_docs = SimpleDirectoryReader(
      input_files=["./data/The_Mindfulness_Toolbox_for_Relationships.pdf"]
  ).load_data(show_progress=True)

  # Build index
  challenging_child_index = VectorStoreIndex.from_documents(challenging_child_docs)
  mindfulness_TB_50_index = VectorStoreIndex.from_documents(mindfulness_TB_50_docs)
  mindfulness_TB_relationships_index = VectorStoreIndex.from_documents(mindfulness_TB_relationships_docs)

  # Persist
  challenging_child_index.storage_context.persist(persist_dir="./storage/challenging_child")
  mindfulness_TB_50_index.storage_context.persist("./storage/mindfulness_TB_50")
  mindfulness_TB_relationships_index.storage_context.persist(persist_dir="./storage/mindfulness_TB_relationships")

INFO:llama_index.indices.loading:Loading all indices.
Loading all indices.
INFO:llama_index.indices.loading:Loading all indices.
Loading all indices.
INFO:llama_index.indices.loading:Loading all indices.
Loading all indices.


In [10]:
challenging_child_engine = challenging_child_index.as_query_engine()
mindfulness_TB_50_engine = mindfulness_TB_50_index.as_query_engine()
mindfulness_TB_relationships_engine = mindfulness_TB_relationships_index.as_query_engine()

### Test to make sure our data loaded

In [11]:
from IPython.display import Markdown

In [12]:
# The_Challenging_Child_Toolbox
query_engine = challenging_child_index.as_query_engine()
response = query_engine.query("What is the definition of a challenging child?")
display(Markdown(f"**Response:**\n\n{response}"))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


**Response:**

A challenging child refers to a child who has emotional and behavioral difficulties. They may require additional support and intervention to address their challenges and help them meet new, higher standards for their actions in daily life.

In [13]:
# The_Mindfulness_Toolbox__50_Practical_Tips_Tools
query_engine = mindfulness_TB_50_index.as_query_engine()
response = query_engine.query("What are the benefits of a mindfulness practice?")
display(Markdown(f"**Response:**\n\n{response}"))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


**Response:**

Mindfulness practice has several benefits. It enhances flexibility and adaptability, allowing individuals to break free from old habits and retrain their brain. It cultivates curiosity and greater ease, encouraging exploration of the journey and process rather than being overly focused on the outcome. Mindfulness also changes one's relationship to self-critical and self-blaming thoughts, promoting greater patience, kindness, acceptance, and hospitality towards oneself and others. Additionally, mindfulness encourages greater fulfillment in daily life by focusing on the present moment, reducing rumination and negative thoughts, as well as anxiety about the future.

In [14]:
# The_Mindfulness_Toolbox_for_Relationships
query_engine = mindfulness_TB_relationships_index.as_query_engine()
response = query_engine.query("How does mindfulness relate to relationships?")
display(Markdown(f"**Response:**\n\n{response}"))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


**Response:**

Mindfulness relates to relationships by engaging a sense of curiosity and openness, which helps individuals escape the limited perspective of the ego. It allows individuals to experience others, even difficult people, in a new way. Mindfulness also helps individuals break free from old habits and toxic ways of reacting in relationships. By cultivating awareness, compassion, and openness, mindfulness provides practical tools for addressing issues that impact daily living and the stresses of the real world. It can uplift and inspire individuals and their relationships, especially in the face of new challenges brought about by technology and social media.

## Define Agent Tools

### Analyze image tool

In [15]:
from llama_index.tools import BaseTool, FunctionTool
from llama_index.multi_modal_llms.gemini import GeminiMultiModal
from llama_index.multi_modal_llms.generic_utils import load_image_urls
from typing import List

In [16]:
def analyze_image(img_urls: List[str]) -> str:
  """Calls our Gemini vision API to analyze the image and return a description of the text contained in the image.
      Returns: A string with a description of the image and the mood it conveys if any.

  Args:
      img_urls (List[str]): The URL of one or more images that convey the users mood
  """
  image_documents = load_image_urls(img_urls)

  gemini = GeminiMultiModal(model="models/gemini-pro-vision", api_key=os.environ.get('GOOGLE_API_KEY'))

  complete_response = gemini.complete(
      prompt="Identify what you see in the image and what mood it conveys if any",
      image_documents=image_documents,
  )

  return complete_response

vision_tool = FunctionTool.from_defaults(fn=analyze_image)

### Mindfulness routine recomendation tool

In [17]:
from llama_index.tools import QueryEngineTool, ToolMetadata

query_engine_tools = [
    QueryEngineTool(
        query_engine=challenging_child_engine,
        metadata=ToolMetadata(
            name="challenging_child",
            description=(
                "The Challenging Child Toolbox 75 Mindfulness Based Practices Tools and Tips for Therapists"
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=mindfulness_TB_50_engine,
        metadata=ToolMetadata(
            name="mindfulness_TB_50",
            description=(
                "The Mindfulness Toolbox 50 Practical Tips Tools Handouts for Anxiety Depression Stress and Pain"
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=mindfulness_TB_relationships_engine,
        metadata=ToolMetadata(
            name="mindfulness_TB_relationships",
            description=(
                "The Mindfulness Toolbox for Relationships 50 Practical Tips Tools Handouts for Building Compassionate Connections"
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]

In [18]:
tools = query_engine_tools + [vision_tool]

## Create LlamaIndex Agent

### System prompt to give our agent some personality

In [19]:
SYSTEM_PROMPT = """You are an emotional support assistant with the expertise of an experienced counselor. Your primary role is to assist the user by encouraging them to provide a drawing that conveys their mood.
You offer professional, friendly, and helpful guidance based on current counseling and mindfulness practices. Once you receive the image, interpret it to discern what the user might be feeling and confirm with them if your observation is correct.
If your interpretation does not align with their feelings, engage in a dialogue until you accurately understand their mood. Your knowledge is exclusively focused on understanding the user's emotions and recommending mindfulness routines using the tool, tailored to their mood.
Thus, you will only provide responses related to these areas. If a question falls outside your area of expertise or if you lack the necessary information, you will inform the user by saying,
'Sorry, I do not know the answer to your question.' and then prompt for more information related to their feelings. Once they confirm that you have correctly understood their feelings, your task is to recommend a suitable mindfulness routine using the tool."""

### Init our agent

In [20]:
from llama_index.agent import OpenAIAgent
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-4-1106-preview") # Using GPT-4 Turbo (Beta)

agent = OpenAIAgent.from_tools(
    tools,
    llm=llm,
    verbose=True,
    system_prompt=SYSTEM_PROMPT,
)

## Test Our Agent

I'm picturing the frontend give the user the option to chat or attach one or more images that would get uploaded to cloud storage.  Then our back could prompt the agent with something like "Here's a drawing of how I'm feeling {Image URL Here}."

For the demo below you need to manually paste the image URL into the chat (i.e. https://ih1.redbubble.net/image.3636044620.1142/bg,f8f8f8-flat,750x,075,f-pad,750x1000,f8f8f8.u5.jpg)

Single prompt example

In [30]:
response = agent.chat('hi')
display(Markdown(f"**Response:**\n\n{response}"))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


**Response:**

Thank you for asking! As an emotional support assistant, I'm here to focus on you. How are you feeling today? If you'd like, you can express your mood through a drawing and share it with me. I'm here to help you interpret your emotions and provide support.

Multi-turn chat example

In [ ]:
#response = agent.chat_repl()

## Initialize Feedback Function(s)

In [32]:
import numpy as np
from trulens_eval import Feedback, Tru, TruLlama
from trulens_eval.feedback import Groundedness
from trulens_eval.feedback.provider.openai import OpenAI

tru = Tru()
# Initialize provider class
openai = OpenAI()

grounded = Groundedness(groundedness_provider=OpenAI())

# Define a groundedness feedback function
f_groundedness = Feedback(grounded.groundedness_measure_with_cot_reasons).on(
    TruLlama.select_source_nodes().node.text.collect()
    ).on_output(
    ).aggregate(grounded.grounded_statements_aggregator)

# Question/answer relevance between overall question and answer.
f_qa_relevance = Feedback(openai.relevance).on_input_output()

# Question/statement relevance between question and each context chunk.
f_qs_relevance = Feedback(openai.qs_relevance).on_input().on(
    TruLlama.select_source_nodes().node.text
    ).aggregate(np.mean)

✅ In groundedness_measure_with_cot_reasons, input source will be set to __record__.app.query.rets.source_nodes[:].node.text.collect() .
✅ In groundedness_measure_with_cot_reasons, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In qs_relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In qs_relevance, input statement will be set to __record__.app.query.rets.source_nodes[:].node.text .


## Instrument app for logging with TruLens

In [33]:
tru_query_engine_recorder = TruLlama(agent,
    app_id='Willow_Alpha',
    feedbacks=[f_groundedness, f_qa_relevance, f_qs_relevance])

INFO:alembic.runtime.migration:Context impl SQLiteImpl.
Context impl SQLiteImpl.
INFO:alembic.runtime.migration:Will assume non-transactional DDL.
Will assume non-transactional DDL.
INFO:trulens_eval.database.sqlalchemy_db:✅ added app Willow_Alpha
✅ added app Willow_Alpha
INFO:alembic.runtime.migration:Context impl SQLiteImpl.
Context impl SQLiteImpl.
INFO:alembic.runtime.migration:Will assume non-transactional DDL.
Will assume non-transactional DDL.
INFO:trulens_eval.database.sqlalchemy_db:✅ added feedback definition feedback_definition_hash_c5f65150c1406850664cb9bd7ade7b5a
✅ added feedback definition feedback_definition_hash_c5f65150c1406850664cb9bd7ade7b5a
INFO:alembic.runtime.migration:Context impl SQLiteImpl.
Context impl SQLiteImpl.
INFO:alembic.runtime.migration:Will assume non-transactional DDL.
Will assume non-transactional DDL.
INFO:trulens_eval.database.sqlalchemy_db:✅ added feedback definition feedback_definition_hash_13cd2005c708c5e4b448579130725fc8
✅ added feedback defini

## Explore in a Dashboard

In [38]:
# or as context manager
with tru_query_engine_recorder as recording:
    agent.query("I am having issues with me wife. Can you provide me with some mindfullness techniques that might help me out?")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
A new object of type <class 'llama_index.memory.chat_memory_buffer.ChatMemoryBuffer'> at 0x269a5b1d7b0 is calling an instrumented method <function ChatMemoryBuffer.put at 0x0000026997DC6320>. The path of this call may be incorrect.
Guessing path of new object is app.memory based on other object (0x269a3c4d720) using this function.
=== Calling Function ===
Calling function: mindfulness_TB_relationships with args: {"input":"I am having issues with my wife and need mindfulness techniques to help me out."}
A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x2699e8a2c20 is calling an instrumented method <function BaseQueryEngine.query at 0x0000026997DC4940>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x2699e8a2d70) u

In [ ]:
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path:   Network URL: http://192.168.0.105:8501



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:alembic.runtime.migration:Context impl SQLiteImpl.
Context impl SQLiteImpl.
INFO:alembic.runtime.migration:Will assume non-transactional DDL.
Will assume non-transactional DDL.
INFO:trulens_eval.database.sqlalchemy_db:✅ feedback result relevance DONE feedback_result_hash_8a1bc08d69de7377f352a993c181349b
✅ feedback result relevance DONE feedback_result_hash_8a1bc08d69de7377f352a993c181349b
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:alembic.runtime.migration:Context impl SQLiteImpl.
Context impl SQLiteImpl.
INFO:alem

In [ ]:
tru.get_records_and_feedback(app_ids=[])[0] # pass an empty list of app_ids to get all

## TODO:

### Reset Agent State

In [ ]:
# agent.reset()

In [ ]:
# response = agent.chat_repl()

### 2nd Session (Returning User)

Specify chat history with a summary of our last session or sessions.  This would need to be queried from our session history.

In [ ]:
# response = agent.chat_repl()